In [98]:
import numpy as np
from rfcm import RFCM
from dtw import dtw
import pandas as pd

In [2]:
class TimeSeries:
    def __init__(self, data: list):
        self.data = np.array(data)
    
    def __add__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data + other.data)
        elif isinstance(other, int):
            return TimeSeries(self.data + other)
        elif isinstance(other, float):
            return TimeSeries(self.data + other)
        elif isinstance(other, np.ndarray):
            return TimeSeries(self.data + other)
        
    def __radd__(self, other):
        return self + other
        
    def __sub__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data - other.data)
        elif isinstance(other, int):
            return TimeSeries(self.data - other)
        elif isinstance(other, float):
            return TimeSeries(self.data - other)
        elif isinstance(other, np.ndarray):
            return TimeSeries(self.data - other)
    
    def __rsub__(self, other):
        return -self + other
    
    def __mul__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data * other.data)
        elif isinstance(other, int):
            return TimeSeries(self.data * other)
        elif isinstance(other, float):
            return TimeSeries(self.data * other)
        elif isinstance(other, np.ndarray):
            return TimeSeries(self.data * other)
    
    def __rmul__(self, other):
        return self * other
        
    def __truediv__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data / other.data)
        elif isinstance(other, int):
            return TimeSeries(self.data / other)
        elif isinstance(other, float):
            return TimeSeries(self.data / other)
        elif isinstance(other, np.ndarray):
            return TimeSeries(self.data / other)
    
    def __floordiv__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data // other.data)
        elif isinstance(other, int):
            return TimeSeries(self.data // other)
        elif isinstance(other, float):
            return TimeSeries(self.data // other)
        elif isinstance(other, np.ndarray):
            return TimeSeries(self.data // other)
        
    def __pow__(self, other):
        if isinstance(other, TimeSeries):
            return TimeSeries(self.data ** other.data)
        elif isinstance(other, int):
            return TimeSeries(self.data ** other)
        elif isinstance(other, float):
            return TimeSeries(self.data ** other)
        elif isinstance(other, np.ndarray):
            return TimeSeries(self.data ** other)
    
    def __neg__(self):
        return TimeSeries(-self.data)
    
    def __repr__(self):
        return str(self.data)
    
    def __len__(self):
        return len(self.data)
    
    def get_data(self):
        return self.data

In [3]:
test_data = np.array([
    [
        TimeSeries([1, 2, 3, 4]),   # 第一個特徵序列
        TimeSeries([1, 1, 1, 1]),   # 第二個特徵序列
        TimeSeries([2, 2, 2, 2])
    ],                  # 第一筆資料
    [
        TimeSeries([4, 3, 2, 1]),
        TimeSeries([4, 4, 4, 4]),
        TimeSeries([3, 3, 3, 3])
    ],                  # 第二筆資料
    [
        TimeSeries([5, 6, 7, 8]),
        TimeSeries([5, 5, 5, 5]),
        TimeSeries([6, 6, 6, 6])
    ],
    [
        TimeSeries([8, 7, 6, 5]),
        TimeSeries([8, 8, 8, 8]),
        TimeSeries([7, 7, 7, 7])
    ]
])
# test_data = np.array([[1, 1], [1, 3], [3, 1], [3, 3], [
#                      9, 9], [9, 11], [11, 9], [11, 11]])
# test_data = np.array([[1,1], [3,3], [9,9], [11,11], [1, 9], [3, 11]])


In [4]:
model = RFCM()
model.fit(test_data)

RFCM()

In [5]:
model.labels_

array([0, 0, 1, 1], dtype=int64)

In [6]:
model.cluster_centers_

array([[[4.71955772 4.74875608 4.77795444 4.80715281],
        [4.71955772 4.71955772 4.71955772 4.71955772],
        [4.74875608 4.74875608 4.74875608 4.74875608]],
       [[4.71955741 4.74898342 4.77840942 4.80783543],
        [4.71955741 4.71955741 4.71955741 4.71955741],
        [4.74898342 4.74898342 4.74898342 4.74898342]]], dtype=object)

In [4]:
def init_memval(n_clusters, n_data):
    U = np.random.random((n_clusters, n_data))
    val = sum(U)
    U = np.divide(U, np.dot(np.ones((n_clusters, 1)),
                    np.reshape(val, (1, n_data))))
    return U

In [6]:
def calc_dtw(data, center):
    dist_part = []
    for center_node in center:
        dist_center_node = []
        for line in data:
            dist_feature = []
            for index in range(len(line)):
                _, cost, _, _ = dtw(line[index].get_data(), center_node[index].get_data(), dist=lambda x, y: np.abs(x - y))
                cost /= len(line[index]) + len(center_node[index])
                dist_feature.append(cost)
            dist_center_node.append(np.linalg.norm(np.array(dist_feature)))
        dist_part.append(dist_center_node)
    return np.array(dist_part)

In [7]:
def center_diff(center, center_old):
    diff = []
    for i in range(len(center)):
        feature = []
        for j in range(len(center[i])):
            feature.append(center[i][j].get_data() - center_old[i][j].get_data())
        diff.append(np.array(feature))
    return np.array(diff)

In [8]:
n_clusters = 2
expo = 2
p = 2
alpha = 2
epsilon = 0.0005
max_iter = 100

np.random.seed(0)
n_data = test_data.shape[0]
U = init_memval(n_clusters, n_data)

In [11]:
def _size_insensitive_rfcm(data, n_clusters, epsilon, expo, max_iter, p):
    """
    Size insensitive object function.

    Args:
        data (array-like, dataframe): Dataset
        n_clusters (int): The number of the clusters
        epsilon (float): The threshold of the convergence
        expo (float): The degree of the fuzziness
        max_iter (int): The maximum number of iterations
        p (int): The fuzziness parameter
    """
    np.random.seed(0)
    n_data = data.shape[0]  # Number of data points
    # Initialize the partition matrix
    U = init_memval(n_clusters, n_data)
    for t in range(max_iter):
        # X[j] is the jth data point, U[i, j] is the membership degree
        mf = np.power(U, expo)
        # The center of the cluster, v_i in the paper
        center = np.divide(
            np.dot(mf, data), (np.ones((data.shape[1], 1))*sum(mf.T)).T)

        # j belongs to A_i,
        # A_i is the set of data points that belong to cluster i
        membership = np.equal(U, U.max(axis=0))
        membership_size = np.sum(membership + np.divide(np.multiply(
            membership, U), np.power(n_data, p)), axis=1)   # According to the paper
        relative_size = np.divide(membership_size, n_data).reshape(
            n_clusters, 1)      # The relative size of the cluster, S_i in the paper

        index_array = np.argmax(U, axis=0)
        interaction_reduction = np.subtract(
            1, np.take_along_axis(relative_size.T, np.expand_dims(index_array, axis=-1), axis=-1)).T  # The interaction reduction, Rho_j in the paper

        # distance part of the U update equation
        dist_part = calc_dtw(data, center) ** (-1 / (expo - 1))

        # coefficient part of the U update equation
        coef_part = np.power(
            1 + np.multiply(membership, np.divide(1, np.power(n_data, p + 1))), (-1 / (expo - 1)))

        # Update the partition matrix
        U = interaction_reduction * np.einsum("ijk->ik", np.divide(
            coef_part[:, None, :] * dist_part, dist_part[:, None, :] * coef_part)) ** -1

        # Check the convergence
        if t > 0:
            if np.linalg.norm(center_diff(center,  center_old)) < epsilon:
                break

        center_old = center

    return U, center

In [12]:
U, center = _size_insensitive_rfcm(test_data, n_clusters, epsilon, expo, max_iter, p)

In [22]:
diff = (calc_dtw(test_data, center) ** (1 / (expo - 1))).T
diff

array([[3.03360221, 4.50122954],
       [1.83220491, 3.2661179 ],
       [1.19072783, 1.47809977],
       [1.8899908 , 0.76891981]])

In [23]:
diff.reshape((test_data.shape[0], 1, -1))

array([[[3.03360221, 4.50122954]],

       [[1.83220491, 3.2661179 ]],

       [[1.19072783, 1.47809977]],

       [[1.8899908 , 0.76891981]]])

In [24]:
diff[:, :, np.newaxis]

array([[[3.03360221],
        [4.50122954]],

       [[1.83220491],
        [3.2661179 ]],

       [[1.19072783],
        [1.47809977]],

       [[1.8899908 ],
        [0.76891981]]])

In [25]:
diff.reshape((test_data.shape[0], 1, -1)).repeat(
    diff.shape[-1], axis=1
)

array([[[3.03360221, 4.50122954],
        [3.03360221, 4.50122954]],

       [[1.83220491, 3.2661179 ],
        [1.83220491, 3.2661179 ]],

       [[1.19072783, 1.47809977],
        [1.19072783, 1.47809977]],

       [[1.8899908 , 0.76891981],
        [1.8899908 , 0.76891981]]])

In [26]:
diff = calc_dtw(test_data, center).T
temp = diff ** (1 / (expo - 1))
denominator_ = temp.reshape((test_data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)
denominator_ = temp[:, :, np.newaxis] / denominator_
# The distance between the data point and the cluster center, S_ij in the paper
dist = (1 / denominator_.sum(2)) ** expo

In [27]:
dist

array([[0.35687424, 0.1620952 ],
       [0.41040161, 0.12914971],
       [0.30673721, 0.19905999],
       [0.08362856, 0.5052565 ]])

In [ ]:
mf = np.power(U, expo)
center = np.divide(
    np.dot(mf, test_data), (np.ones((test_data.shape[1], 1))*sum(mf.T)).T)
print("\nmf")
print(mf)

print("\ncenter")
print(center)

In [ ]:
membership = np.equal(U, U.max(axis=0))
membership_size = np.sum(membership + np.divide(np.multiply(membership, U), np.power(n_data, p)), axis=1)
relative_size = np.divide(membership_size, n_data).reshape(n_clusters, 1)

print("\nmembership")
print(membership)

print("\nmembership_size")
print(membership_size)

print("\nrelative_size")
print(relative_size)

In [ ]:
index_array = np.argmax(U, axis=0)
interaction_reduction = np.subtract(
    1, np.take_along_axis(relative_size.T, np.expand_dims(index_array, axis=-1), axis=-1)).T
print("\ninteraction_reduction")
print(interaction_reduction)

In [ ]:
dist_part = []
for center_node in center:
    dist_center_node = []
    for line in test_data:
        dist_feature = []
        for index in range(len(line)):
            _, cost, _, _ = dtw(line[index].get_data(), center_node[index].get_data(), dist=lambda x, y: np.abs(x - y))
            cost /= len(line[index]) + len(center_node[index])
            dist_feature.append(cost)
        dist_center_node.append(np.linalg.norm(np.array(dist_feature)))
    dist_part.append(dist_center_node)
dist_part = np.array(dist_part)
print("\ndist_part")
print(dist_part)

In [ ]:
# dist_part = (np.sqrt(np.einsum("ijk->ij", (test_data[:, None, :] - center) ** 2))).T
# print("\ndist_part")
# print(dist_part)
coef_part = 1 + np.multiply(membership, np.divide(1, np.power(n_data, p + 1)))
print("\ncoef_part")
print(coef_part)

# np.einsum("ijk->jk", np.divide(coef_part[:, None, :] * dist_part, dist_part[:, None, :] * coef_part))
# interaction_reduction * np.divide(numerator_part, denominator_part) ** -1

In [ ]:
np.einsum("ijk->jk", np.divide(coef_part[:, None, :] * dist_part, dist_part[:, None, :] * coef_part))

In [ ]:
np.divide(coef_part[:, None, :] * dist_part, dist_part[:, None, :] * coef_part)

In [ ]:
np.einsum("ijk->ik", np.divide(coef_part[:, None, :] * dist_part, dist_part[:, None, :] * coef_part))

In [237]:
mf = np.power(U, expo)
center = np.divide(
    np.dot(mf, test_data), (np.ones((test_data.shape[1], 1))*sum(mf.T)).T)
print("\ncenter")
print(center)

membership = np.equal(U, U.max(axis=0))
membership_size = np.sum(membership + np.divide(np.multiply(membership, U), np.power(n_data, p)), axis=1)
relative_size = np.divide(membership_size, n_data).reshape(n_clusters, 1)

index_array = np.argmax(U, axis=0)
interaction_reduction = np.subtract(
    1, np.take_along_axis(relative_size.T, np.expand_dims(index_array, axis=-1), axis=-1)).T

# dist_part = (np.sqrt(
#     np.einsum("ijk->ij", (test_data[:, None, :] - center) ** 2)) ** (-1 / (expo - 1))).T
dist_part = calc_dtw(test_data, center)
coef_part = np.power(
    1 + np.multiply(membership, np.divide(1, np.power(n_data, p + 1))), (-1 / (expo - 1)))

U = interaction_reduction * np.einsum("ijk->ik", np.divide(coef_part[:, None, :] * dist_part, dist_part[:, None, :] * coef_part)) ** -1

try:
    print("\nnorm")
    print(np.linalg.norm(center_diff(center, center_old)))

    if np.linalg.norm(center_diff(center,  center_old)) < epsilon:
        print("\ncenter converged")
except:
    pass
    
center_old = center



center
[[[4.48431379 4.49787121 4.51142863 4.52498605]
  [4.48431379 4.48431379 4.48431379 4.48431379]
  [4.49787121 4.49787121 4.49787121 4.49787121]]
 [[4.51568621 4.50212879 4.48857137 4.47501395]
  [4.51568621 4.51568621 4.51568621 4.51568621]
  [4.50212879 4.50212879 4.50212879 4.50212879]]]

norm
0.0004560959705128383

center converged


In [ ]:
np.multiply(membership, np.divide(1, np.power(n_data, p + 1)))

In [ ]:
np.expand_dims(index_array, axis=-1)

In [ ]:
relative_size.T

In [ ]:
np.take_along_axis(relative_size.T, np.expand_dims(index_array, axis=-1), axis=-1)

In [ ]:
center

In [ ]:
cluster_handler = RFCM()

In [ ]:
cluster_handler.fit(test_data)

In [ ]:
cluster_handler.labels_

In [ ]:
def init_memval(n_clusters, n_data):
        U = np.random.random((n_clusters, n_data))
        val = sum(U)
        U = np.divide(U, np.dot(np.ones((n_clusters, 1)),
                        np.reshape(val, (1, n_data))))
        return U

In [ ]:
def _exp_func(diff, omega):
    return 1 - np.exp(-diff / omega)

def _exp_derivative_func(diff, omega):
    return (1 / omega) * np.exp(-diff / omega)

In [ ]:
mf = np.power(U, expo)
center = np.divide(
    np.dot(mf, data), (np.ones((data.shape[1], 1))*sum(mf.T)).T)

In [ ]:
diff = np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2))
temp = diff ** (1 / (expo - 1))
denominator_ = temp.reshape((data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)
denominator_ = temp[:, :, np.newaxis] / denominator_

In [ ]:
dist = (1 / denominator_.sum(2)) ** expo

In [ ]:
omega = sum(dist * diff) / (alpha * sum(dist))

In [ ]:
dist_part = (_exp_func(diff, omega) ** (-1 / (expo - 1))).T

In [ ]:
coef_part = np.ones_like(U)

In [ ]:
numerator_part = np.dot(np.ones((n_clusters, 1)), np.reshape(
    sum(coef_part), (1, coef_part.shape[1]))) * dist_part
denominator_part = np.dot(np.ones((n_clusters, 1)), np.reshape(
    sum(dist_part), (1, dist_part.shape[1]))) * coef_part

In [ ]:
membership = np.equal(U, U.max(axis=0))
dist_part = (np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2)) ** (-1 / (expo - 1))).T
coef_part = np.power(1 - np.multiply(membership, np.divide(1, np.power(n_data, p + 1))), (-2 / (expo - 1)))

In [ ]:
coef_part

In [ ]:
np.dot(np.ones((n_clusters, 1)), np.reshape(
                sum(coef_part), (1, coef_part.shape[1])))

In [ ]:
data = a

In [ ]:
def _exp_derivative_func(diff, omega):
    return (1 / omega) * np.exp(-diff / omega)

In [ ]:
U

In [ ]:
np.equal(U, U.max(axis=0))

In [ ]:
np.argmax(U, axis=0)

In [ ]:
mf = np.power(U, expo)
center = np.divide(
    np.dot(mf, data), (np.ones((data.shape[1], 1))*sum(mf.T)).T)

In [ ]:
center

In [ ]:
diff = np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2))
temp = diff ** (1 / (expo - 1))
denominator_ = temp.reshape((data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)
denominator_ = temp[:, :, np.newaxis] / denominator_
dist = (1 / denominator_.sum(2)) ** expo

In [ ]:
omega = sum(dist * diff) / (2 * sum(dist))

In [ ]:
mf.T * _exp_derivative_func(diff, omega)

In [ ]:
center

In [ ]:
temp

In [ ]:
dist_part

In [ ]:
diff = np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2))
temp = diff ** (1 / (expo - 1))
denominator_ = temp.reshape((data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)
denominator_ = temp[:, :, np.newaxis] / denominator_
dist = (1 / denominator_.sum(2)) ** expo

In [ ]:
dist

In [ ]:
diff

In [ ]:
sum(dist * diff) / (alpha * sum(dist))

In [ ]:
temp.reshape((data.shape[0], 1, -1)).repeat(
    temp.shape[-1], axis=1
)

In [ ]:
denominator_.sum(2)

In [ ]:
diff = np.zeros((center.shape[0], data.shape[0]))
if center.shape[1] > 1:
    for k in range(center.shape[0]):
        diff[k, :] = np.sqrt(sum(np.power(
            data-np.dot(np.ones((data.shape[0], 1)), np.reshape(center[k, :], (1, center.shape[1]))), 2).T))
else:  # for 1-D data
    for k in range(center.shape[0]):
        diff[k, :] = abs(center[k]-data).T
# dist = diff + 0.0001
dist_part = diff ** (-2 / (expo - 1))


In [ ]:
membership = np.equal(U, U.max(axis=0))
dist_part = (np.sqrt(np.einsum("ijk->ij", (data[:, None, :] - center) ** 2)) ** (-2 / (expo - 1))).T
coef_part = np.power(1 - np.multiply(membership, np.divide(1, np.power(n_data, p + 1))), (-2 / (expo - 1)))

In [ ]:
numerator_part = np.dot(np.ones((n_clusters, 1)), np.reshape(
                sum(coef_part), (1, coef_part.shape[1]))) * dist_part

In [ ]:
np.dot(np.ones((n_clusters, 1)), np.reshape(
                sum(dist_part), (1, dist_part.shape[1]))) * coef_part

In [ ]:
membership = np.equal(U, U.max(axis=0))
membership_size = np.divide(
    np.sum(np.multiply(membership, U), axis=1), np.power(n_data, p))
relative_size = np.divide(
                np.add(membership_size, 1), n_data).reshape(n_clusters, 1)

In [ ]:
index_array = np.argmax(U, axis=0)
interaction_reduction = np.subtract(
    1, np.take_along_axis(relative_size.T, np.expand_dims(index_array, axis=-1), axis=-1)).T

In [ ]:
interaction_reduction

In [ ]:
data = np.array([[1, 2], [1, 4], [1, 0], [4, 2], [4, 4], [4, 0]])
n_data = data.shape[0] # Number of data points
n_clusters = 3
p = 2
expo = 2


In [ ]:
# Initialize the partition matrix
U = np.random.random(size=(n_clusters, n_data))
val = sum(U)
U = np.divide(U, np.dot(np.ones((n_clusters, 1)), np.reshape(val, (1, n_data))))


In [ ]:
mf = np.power(U, expo)
np.divide(np.dot(mf, data), (np.ones((data.shape[1], 1))*sum(mf.T)).T)

In [ ]:
np.array([np.sum(data[j, :] * U[i, j] ** 2 for j in range(n_data))
         for i in range(n_clusters)])


In [ ]:
np.sum(1 + U[0, j] / (X.size ** p) for j in range(n))

In [ ]:
S.append(sum([1 + u(i, j) / pow(n, p) for i in range(n_clusters) for j in range(n)]) / X.size)

In [ ]:
v = np.empty((n_clusters, X.shape[1])) # The cluster centers

In [ ]:
v[0] = np.sum([U[0, j] ** 2 * X[j] for j in range(n)])

In [ ]:
[X[j,:] * U[0, j] ** 2 for j in range(n)]

In [ ]:
np.sum(np.array([[1, 2], [1, 4]]), axis=0)

In [ ]:
np.array([np.sum(X[j,:] * U[0, j] ** 2 for j in range(n)) for i in range(3)])